<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agent_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive LLM Agent Dealing with Web Info

FLAML offers an experimental feature of interactive LLM agents, which can be used to solve various tasks, including coding and math problem-solving.

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to discuss a paper based on its URL. Here `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. By setting `human_input_mode` properly, the `UserProxyAgent` can also prompt the user for feedback to `AssistantAgent`. For example, when `human_input_mode` is set to "ALWAYS", the `UserProxyAgent` will always prompt the user for feedback. When user feedback is provided, the `UserProxyAgent` will directly pass the feedback to `AssistantAgent` without doing any additional steps. When no user feedback is provided, the `UserProxyAgent` will execute the code written by `AssistantAgent` directly and returns the execution results (success or failure and corresponding outputs) to `AssistantAgent`.

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [openai] option:
```bash
pip install flaml[autogen]
```

In [1]:
# %pip install flaml[autogen]

## Set your API Endpoint

The [`config_list_openai_aoai`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_openai_aoai) function tries to create a list of configurations using Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.

The following code excludes openai endpoints from the config list.
Change to `exclude="aoai"` to exclude Azure OpenAI, or remove the `exclude` argument to include both.


In [2]:
from flaml import oai

config_list = oai.config_list_openai_aoai(exclude="openai")

## Construct Agents

We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for feedback when it receives a "TERMINATE" signal from the assistant agent.

In [3]:
from flaml.autogen.agent.assistant_agent import AssistantAgent
from flaml.autogen.agent.user_proxy_agent import UserProxyAgent

# create an assistant which is essentially a AssistantAgent instance named "coding_agent"
assistant = AssistantAgent(
    name="assistant",
    request_timeout=600,
    seed=42,
    config_list=config_list,
    model="gpt-4-32k",  # make sure the endpoint you use supports the model
)
# create a UserProxyAgent instance named "user"
user = UserProxyAgent(
    name="user",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.rstrip().endswith("TERMINATE"),
    work_dir='web',
)

## Perform a task

We invoke the `receive()` method of the coding agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after receving a message from the coding agent. If you don't provide any feedback (by pressing Enter directly), the user proxy agent will try to execute the code suggested by the coding agent on behalf of you, or terminate if the coding agent sends a "TERMINATE" signal in the end of the message.

In [4]:
# the assistant receives a message from the user, which contains the task description
assistant.receive(
    """
What is a good followup study for this paper: https://arxiv.org/abs/2303.04673
""",
    user
)


**** assistant received message from user ****


What is a good followup study for this paper: https://arxiv.org/abs/2303.04673


**** user received message from assistant ****

To get the abstract of the paper and suggest a good follow-up study, let's write a Python script to extract the abstract and analyze it. Please run the following Python code:

```python
import requests
from bs4 import BeautifulSoup

url = 'https://arxiv.org/abs/2303.04673'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
abstract = soup.find('blockquote', {'class': 'abstract'}).text.strip().replace('\n', ' ')

print(f'Abstract: {abstract}')

# Analyze the abstract and suggest a follow-up study
```

After you run the code, I'll analyze the abstract and help you find a good follow-up study for the paper.

>>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...

**** assistant received message from user ****

exitcode: 0 (execution succeeded)
Code output: Abstract: Abstr